# Data Export with the lucullus REST API

## Introduction

Hello dear reader! This example file shold guide you through all the steps necessary to export data through the Lucullus REST API using the "lucullus_rest" python library. With this you can easily export data without having to open Lucullus, and there are several reasons why you would like to do this:
* Automation saves time: If you know exactly what you need, and if you always need the same data, it can save some time to automatically export your data, especially if you need to extract data of many processes. Setting up a script can be especially useful if you need to export data of many processes. Just imagine you forgot about exporting a signal, in which case you only need to update your script instead of manually extracting all the data on your own again.
* Familiarizing yourself with how REST works will help you also with other interfaces for collecting data.

## Getting process data by hand

At first, to show how things work, we will get the data through the REST API step by step. For this we need to import the "requests" module to send and recieve data from the web. We will define our username and password so we can authenticate ourselfes and we want to get the pO2 signal from process PIP_555.

In [1]:
import requests

auth = ("user", "password") # Change user and password!!!
process = "Process_555"
port_name = "PV_Temp"

To collect this signal we need to get some more information. In Lucullus, every object is defined through a specific ID, so a process has a process ID, a reactor has a reactor ID, a port has a port ID and so on. At first we will find out what the process ID of our process is. For this we will create a URL that tells the REST API that we want to find a process which name is "PIP_555", and then we will get this information. 

In [2]:
rest_url = "http://XXX.XXX.XXX.XXX:8080/lpims/rest/v1/"

request_link = rest_url + "processes?name={}".format(process)
response = requests.get(request_link, auth=auth)
print(response)

<Response [200]>


The status code of our response should be "200" which tells us that our request was valid and that we could retrieve our information. From the response, we can get the information that we recieved in json format with the .json() command, which returns us a dictionary that we can navigate. From there we can get the information which process id defines our process.

In [3]:
process_id = response.json()["data"][0]["id"]
print(process_id)

1065


The same we also have to do for our port.

In [4]:
request_link = rest_url + "ports?name={}".format(port_name)
response = requests.get(request_link, auth=auth)

port_id = response.json()["data"][0]["id"]
print(port_id)

268


Now that we know our process and port Id we can get our data. The sensor data is stored as a "signal" (which again has a dedicated signal ID, you get the point). A signal is the data of a specific process for a specific port. We can get the signals by telling the REST API to search for signals by matching process and port ID. From there on we get again a json file, from which we can get the time-value pairs as a loooong list.

In [5]:
request_link = rest_url+"signals?processId={}&portId={}".format(process_id, port_id)
response = requests.get(request_link, auth=auth)

signal_data = response.json()["data"]["values"]
print(signal_data[0:10])

[[0.0014952180555555556, 73.20916748046875], [0.002919173055555555, 73.23304748535156], [0.004308261388888889, 73.68672180175781], [0.005727940833333334, 73.20916748046875], [0.007117025, 73.18528747558594], [0.00850617, 73.63896942138672], [0.009936127777777777, 73.80611419677734], [0.01132524222222222, 80.39636993408203], [0.012714335277777778, 76.12225341796875], [0.014103432777777777, 75.62081909179688]]


## Using the internal lucullus_rest module

Of course, to get the signal data in this way required a bit work and might not be very convenient for day to day use. As a consequence we created a Python module named lucullus_rest that tries to make things easier for us. From this module we can import the function export_to_df which takes a process name, a list of port names and authentication information as input, and returns to us a pandas dataframe with all the data. The data is in the same format if you would have exported the process data as an excel file through the graphic tool, which means that existing workflows do not have to be changed.

In [7]:
from lucullus_rest.core import export_to_df

auth = ("user", "password")
process = "Process_555"
port_names = ["PV_pO2", "PV_Temp"]

df = export_to_df(process, port_names, auth)
print(df)

         Time [h]    PV_Temp     PV_pO2
0        0.001495  17.876518  73.209167
1        0.002919  17.876518  73.233047
2        0.004308  17.865667  73.686722
3        0.005728  17.892794  73.209167
4        0.007117  17.919922  73.185287
...           ...        ...        ...
87244  121.179309  51.003689  56.519730
87245  121.180698  51.258678  56.635124
87246  121.182087  51.486542  56.542809
87247  121.183476  51.578773  56.542809
87248  121.184865  51.795788  57.119778

[87249 rows x 3 columns]


In the previous examples we got ALL time-value pairs, but maybe we only want the data interpolated for all 5 minutes. In this case we can do the same thing, but specify an interpolation interval of 300 seconds.

In [8]:
df = export_to_df(process, port_names, auth, interval=300)
print(df)

        Time [h]    PV_Temp      PV_pO2
0       0.001495  17.876518   73.209167
1       0.084829  27.587854  103.868195
2       0.168162  29.257763  103.898261
3       0.251495  29.855684  104.032183
4       0.334829  30.300564  103.415624
...          ...        ...         ...
1450  120.834829  22.734263   56.983755
1451  120.918162  21.454226   56.960584
1452  121.001495  21.604972   57.217340
1453  121.084829  20.716512   56.804750
1454  121.168162  49.530108   57.126532

[1455 rows x 3 columns]


Now that we have extracted the data, we can easily save it as an excel to a folder of our choice.

In [9]:
save_folder = "C:/Users/Public/Documents"
save_path = save_folder + "/{}.xlsx".format(process)
df.to_excel(save_path)

If we wanted to extract now the data for a number of processes, we could of course also do it, for instance like this:

In [35]:
list_of_processes = ["Process_555", "Process_556"]

for p in list_of_processes:
    df = export_to_df(p, port_names, auth)
    save_path = save_folder + "/{}.xlsx".format(p)
    df.to_excel(save_path)